In [3]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import pickle
from sklearn.manifold import TSNE, LocallyLinearEmbedding
from scipy.spatial.distance import pdist, squareform
from scipy import signal
# import skvideo.io
from openTSNE import TSNE as oTSNE
from openTSNE import  affinity
import math
import cv2
import os
from tqdm.notebook import trange, tqdm
plt.switch_backend('Agg')

from IPython.display import display, Image
import ipywidgets as widgets
import threading
import multiprocessing as mp



In [4]:
def calc_dist(data):
    dist = 1 - np.abs(np.corrcoef(data, rowvar=True))
    # eliminating pixels with nan distance. These are pixels who have not changed across all images. That is std = 0
    np.nan_to_num(dist, copy=False)
    eleminated_pixel_idx = np.all(np.isnan(dist), axis=0)
    dist = dist[:, ~np.all(np.isnan(dist), axis=0)]
    dist = dist[~np.all(np.isnan(dist), axis=1), :]
    return dist, eleminated_pixel_idx

def plot_frame(frame, pixel_pos):
    fig, ax = plt.subplots(1, 1, sharey=False, figsize=(1.5, 1.5), dpi=200)
    # ax.scatter(positions_embd[:, 0], positions_embd[:, 1], s=4, c=test_image[~bad_pixel_idx])
    frame_colors = frame.reshape(-1, 3) / 255 
    # print('colors shape', frame_colors.shape)
    # ax.scatter(pixel_pos[~bad_pixel_idx, 0], pixel_pos[~bad_pixel_idx, 1], s=4, c=frame_colors[~bad_pixel_idx], alpha=0.5)
    ax.scatter(pixel_pos[:, 0], pixel_pos[:, 1], s=4, c=frame_colors[:], alpha=0.5)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-4, 4)
    ax.set_facecolor("black")
    fig.canvas.draw()
    plot_data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    plt.close()
    return plot_data.reshape(fig.canvas.get_width_height()[::-1] + (3,))


def tsne_proc(q_send, q_receive):
    positions_embd = None
    while not q_receive.empty():
        dist_mat = q_receive.get()
        positions_embd = oTSNE(n_components=2, verbose=False, metric="precomputed", perplexity=14.0, initialization='random' if positions_embd is None else positions_embd ).prepare_initial(dist_mat)    
        positions_embd = positions_embd.optimize(n_iter=10, inplace=False, exaggeration=12 , n_jobs=-1)        
        q_send.put(positions_embd)


In [8]:
# Stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

freezeButton = widgets.ToggleButton(
    value=False,
    description='Freeze',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='pause' # (FontAwesome names without the `fa-` prefix)
)


# Display function
# ================
def view(button,):
    cap = cv2.VideoCapture(0)
    # cap.set(3,64) # adjust width
    # cap.set(4,48) # adjust height
    display_handle=display(None, display_id=True)
    i = 0
    # Need a running window of frames to calculate distances
    window_size = 100
    frames = []
    positions_embd = None
    flip_flop_x = 0
    flip_flop_y = 0

    first_iter = True


    print('Looppingggggggg')
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1)[flip_flop_x::24, flip_flop_y::24] # if your camera reverses your image
        # flip_flop_x = 1 - flip_flop_x
        # flip_flop_y = 1 - flip_flop_y if flip_flop_x == 0 else flip_flop_y

        
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
        if len(frames) >= window_size:  
            

            frames_flat = np.stack(frames, axis=-1).reshape(-1, window_size)  
            dist_mat, bad_pixel_idx = calc_dist(frames_flat)   


                
            if freezeButton.value==False:
                positions_embd = oTSNE(n_components=2, verbose=False, metric="precomputed", perplexity=14.0, initialization='random' if positions_embd is None else positions_embd ).prepare_initial(dist_mat)    
                positions_embd = positions_embd.optimize(n_iter=10, inplace=False, exaggeration=12 , n_jobs=-1)
            
            plotted_frame = plot_frame(frame, positions_embd)
            _, frame_jpeg = cv2.imencode('.jpeg', plotted_frame)

            display_handle.update(Image(data=frame_jpeg.tobytes()))
            
            frames.pop(0)
            
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)

            break

            
# Run
# ================

# ctx = mp.get_context('fork')    
# q_receive = ctx.SimpleQueue()
# q_send = ctx.SimpleQueue()
# p = ctx.Process(target=tsne_proc, args=(q_receive, q_send))
# p.start()

display(stopButton)
display(freezeButton)
thread = threading.Thread(target=view, args=(stopButton, ))
thread.start()
            # p.join()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

ToggleButton(value=False, button_style='warning', description='Freeze', icon='pause', tooltip='Description')

[ WARN:0@51.432] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


None

Looppingggggggg


Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may have display poor convergence.
Standard deviation of embedding is greater than 0.0001. Initial embeddings with high variance may

In [5]:
cap = cv2.VideoCapture(0)
cap.set(3,640) # adjust width
cap.set(4,480) # adjust height

success, img = cap.read()


[ WARN:0@2.305] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [3]:
import multiprocessing as mp

def foo(q):
    q.put('hello')

if __name__ == '__main__':
    ctx = mp.get_context('fork')
    q = ctx.Queue()
    p = ctx.Process(target=foo, args=(q,))
    p.start()
    print(q.get())
    p.join()

hello


In [ ]:
from multiprocessing import Process, Manager

def worker(shared_list):
    shared_list.append('Worker')

if __name__ == '__main__':
    manager = Manager()
    shared_list = manager.list()

    p = Process(target=worker, args=(shared_list,))
    p.start()
    p.join()

    print(shared_list)